<center>

# DECA: Detailed Expression Capture and Animation

Code in [![GitHub stars](https://img.shields.io/github/stars/yadiraf/DECA?style=social)](https://github.com/YadiraF/DECA)

Page at [![](https://img.shields.io/badge/Project-Page-blue?style=flat&logo=Google%20chrome&logoColor=blue)](https://deca.is.tue.mpg.de/)

Made by [![Yao](https://img.shields.io/twitter/follow/yaofeng1995?style=social)](https://twitter.com/yaofeng1995)

</center>

# **安装环境**

In [ ]:
#@title 克隆git仓库（Done）
# %cd C://Users//Administrator//Desktop
# !git clone https://github.com/Jimmy142857/DECA_Analyze.git

In [ ]:
#@title Dependencies List (Skip this Step and start from the next)
#@markdown GPU is needed

# !pip install -r requirements.txt
# !pip install 'torch==1.6.0'
# !pip install 'torchvision==0.7.0'
# !pip install -q 'pytorch3d==0.2.5'
# !pip install -q numpy scipy chumpy scikit-image opencv-python
# !pip install -q PyYAML face-alignment yacs kornia ninja fvcore
# !pip install -q lucid>=0.2.3 gdown matplotlib
# !pip install --upgrade ipykernel

In [ ]:
#@title 下载模型 (Done)
#@markdown By executing this cell, you agree to the [LICENSE](https://flame.is.tue.mpg.de/modellicense.html) provided by Max-Planck-Gesellschaft zur Förderung der Wissenschaften e.V

print("Downloading FLAME2020 model...")              # 下载FLAME模型
!gdown --id 18eHkbq2W3aJJVUNTM3QwFK0dPaeNoRAz -O FLAME2020.zip
!unzip -o FLAME2020.zip -d data/

print("Downloading deca_model...")                 # 下载训练好的DECA参数
!gdown --id 1rp8kdyLPvErw2dTmqtjISRVvQLj6Yzje -O data/deca_model.tar

# **开始,需要GPU**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 安装库
!apt -q install -y zip unzip ffmpeg libsm6 libxext6
!pip install pytorch3d scipy chumpy scikit-image opencv-python
!pip install PyYAML face-alignment yacs kornia ninja fvcore lucid gdown

In [ ]:
!nvidia-smi
%cd /content/drive/MyDrive/DECA/

# **基本 Reconstruction**


In [ ]:
#@title 运行 Demo 中的图片

print("Running experiments...")
import os
input_folder = "TestSamples/AFLW2000" #@param {type:"string"}
output_folder = os.path.join(input_folder, "results")
!python demos/demo_reconstruct.py -i $input_folder -s $output_folder --saveDepth True --saveObj True


In [ ]:
#@title 使用自己的图片
#@markdown The size of your picure should be limited

print("Running experiments...")
import os

input_folder = "/content/drive/MyDrive/Face_plasy" #@param {type:"string"}
output_folder = os.path.join(input_folder, "results")
!python demos/demo_reconstruct.py -i $input_folder -s $output_folder --saveDepth True --saveObj True --saveKpt True


# show results 
# print('visualize one exmaple below')
# import matplotlib.pyplot as plt
# from glob import glob
# from PIL import Image

# vispath_list = glob(output_folder+'/*_size.jpg')
# for vispath in vispath_list:
#   image = Image.open(vispath)
#   plt.figure(figsize=(20, 20))
#   plt.imshow(image)
#   plt.axis("off");
#   break
# print(f'Please check all results in {output_folder}')


In [ ]:
#@title 下载结果
import os
from google.colab import files

print(next(os.walk(output_folder)))
folders = [ os.path.join(output_folder, f) for f in next(os.walk(os.path.join(input_folder, 'results')))[1] ]

print(f'Download results...')
os.system(f'zip -r DECA_results.zip {" ".join(folders)}')
files.download("DECA_results.zip")

# **Expression Transfer**

In [ ]:
#@title 表情迁移
image_path = '/content/drive/MyDrive/Face_jimmy_abnormal/jimmy_1.jpg'
exp_path = '/content/drive/MyDrive/DECA/TestSamples/exp/7.jpg'
savefolder = ''

# --image_path $image_path --exp_path $exp_path

!python demos/demo_transfer.py --saveObj true --image_path $image_path --exp_path $exp_path

# **Reposing and Animation**

In [ ]:
#@title Reposing and Animation (Error 缺少文件FLAME_albedo_from_BFM.npz)
!python demos/demo_teaser.py 

# **Trainning!!!**

In [ ]:
#@title 尝试训练（Face Segmentation 步骤未完成）
%cd /content/drive/MyDrive/DECA

# !python main_train.py --cfg configs/release_version/deca_pretrain.yml # 使用已有的预训练模型
!python main_train.py --cfg configs/release_version/deca_coarse.yml 
!python main_train.py --cfg configs/release_version/deca_detail.yml 

# **代码解读**

In [ ]:
%cd /content/drive/MyDrive/DECA

In [ ]:
# yaml为配置文件同yac库一起使用
import yaml
f = open(r'/content/drive/MyDrive/DECA/configs/release_version/deca_pretrain.yml')
y = yaml.load(f, Loader=yaml.FullLoader)
print (y.keys())

In [ ]:
# mask模板操作
from skimage.io import imread
import numpy as np
import torch

face_eye_mask_path = '/content/drive/MyDrive/DECA/data/uv_face_eye_mask.png'
mask = imread(face_eye_mask_path).astype(np.float32)/255.
mask = torch.from_numpy(mask[:,:,0])[None,None,:,:].contiguous()

In [ ]:
# displacement map查看
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

# np.set_printoptions(threshold=np.inf)
displacement_path = '/content/drive/MyDrive/DECA/data/fixed_displacement_256.npy'
disp = np.load(displacement_path)
disp = disp

imshow(disp, cmap="gray")
plt.axis('off')
plt.show()

In [ ]:
# 密集网格模板查看，转为txt保存
import torch
import numpy as np

np.set_printoptions(threshold=np.inf)  # 解决输出不完整问题
dense_template_path = '/content/drive/MyDrive/DECA/data/texture_data_256.npy'
dense_txt = '/content/drive/MyDrive/DECA/data/texture_data_256.txt'

dense_template = np.load(dense_template_path, allow_pickle=True, encoding='latin1').item()
# with open(dense_txt, 'w') as dense:
#   for k,v in dense_template.items():
#     dense.write(k + ':' + str(v))
#     dense.write('\n')

print(type(dense_template))

In [ ]:
# 六参数维度保存

model = { 
  'n_shape':100,
  'n_tex':50,
  'n_exp' : 50,
  'n_cam': 3,
  'n_pose': 6,
  'n_light' : 27,
  }

param_list = ['shape', 'tex', 'exp', 'pose', 'cam', 'light']
param_dict = {i:model.get('n_' + i) for i in param_list}
print(param_dict)

{'shape': 100, 'tex': 50, 'exp': 50, 'pose': 6, 'cam': 3, 'light': 27}
